# Evn

In [ ]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
# gpu 사용량 확인
!nvidia-smi

In [ ]:
# data dir
data_dir = '/content/drive/MyDrive/문서/강의계획서/삼성전기/삼성전기.20220228/data'
os.listdir(data_dir)

# Word2Vec

## Tutorial

In [ ]:
# 입력 문장
sentences = [
    "나는 학생 입니다",
    "당신은 수학 선생님 입니다",
    "나는 선생님 입니다",
    "당신은 수학 학생 입니다"
]

In [ ]:
words = []
for line in sentences:
    for w in line.split():
        words.append(w)

words = list(dict.fromkeys(words))
words

In [ ]:
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    if w not in word_to_id:
        word_to_id[w] = len(word_to_id)
word_to_id

In [ ]:
id_to_word = {_id:w for w, _id in word_to_id.items()}
id_to_word

In [ ]:
# 입력(c), 출력(o) 정의
inputs = np.array([word_to_id['나는']])
labels = np.array([word_to_id['학생']])
inputs, labels

In [ ]:
# center word embedding
V = tf.keras.layers.Embedding(len(word_to_id), 4)
v = V(inputs)
v

In [ ]:
# weight of center matrix
V.get_weights()

In [ ]:
# outer word embedding transpose
U = tf.keras.layers.Dense(len(word_to_id), use_bias=False)
vu = U(v)
vu

In [ ]:
# weight of outer matrix
U.get_weights()[0].shape, U.get_weights()

In [ ]:
# exp(uv) / sum(exp(uv))
vu_prob = tf.nn.softmax(vu)
vu_prob

In [ ]:
# outer word embedding transpose & softmax
U = tf.keras.layers.Dense(len(word_to_id), use_bias=False, activation=tf.nn.softmax)
y_pred = U(v)
y_pred

In [ ]:
# weight of outer matrix
U.get_weights()[0].shape, U.get_weights()

In [ ]:
# label one hot
y_true = tf.one_hot(labels, len(word_to_id))
y_true

In [ ]:
# cross entroyp loss
loss = - y_true * tf.math.log(y_pred)
loss, tf.reduce_sum(loss, axis=1)

In [ ]:
# sparse_categorical_crossentropy
tf.keras.losses.sparse_categorical_crossentropy(labels, y_pred)

In [ ]:
def build_model(n_vocab, d_model):
    """
    skim gram 학습 모델
    :param n_vocab: vocabulary 단어 수
    :param d_model: 단어를 의미하는 벡터의 차원 수
    """
    tokens = tf.keras.layers.Input(shape=(1,))

    # center word vector
    V = tf.keras.layers.Embedding(n_vocab, d_model)
    v = V(tokens)
    # 단어 예측 (activation=tf.nn.softmax)
    U = tf.keras.layers.Dense(n_vocab, use_bias=False, activation=tf.nn.softmax)
    y_pred = U(v)

    model = tf.keras.Model(inputs=tokens, outputs=y_pred)
    return model

In [ ]:
# 모델 생성
model = build_model(len(word_to_id), 32)
# 모델 내용 그래프 출력
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

## Data

In [ ]:
# 학습할 말뭉치
text = """수학은 수식이 복잡해서 어렵다
수학은 공식이 많아서 어렵다
수학은 수식이 이해되면 쉽다
수학은 공식이 능통하면 쉽다
영어는 단어가 많아서 어렵다
영어는 듣기가 복잡해서 어렵다
영어는 단어가 이해되면 쉽다
영어는 듣기가 능통하면 쉽다
국어는 지문이 복잡해서 어렵다
국어는 한문이 많아서 어렵다
국어는 지문이 이해되면 쉽다
국어는 한문이 능통하면 쉽다"""

## vocabulary

In [ ]:
words = []
for line in text.split("\n"):
    for w in line.split():
        words.append(w)

words = list(dict.fromkeys(words))
# words = dict.fromkeys(words)
words

In [ ]:
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    if w not in word_to_id:
        word_to_id[w] = len(word_to_id)
word_to_id

In [ ]:
id_to_word = {_id:w for w, _id in word_to_id.items()}
id_to_word

## Train 데이터 생성

In [ ]:
def make_data(src, n_win):
    inputs, labels = [], []

    for s in src:
        tokens = s.split()
        for i, c in enumerate(tokens):
            left = max(i - n_win, 0)
            right = min(i + n_win, len(tokens) - 1)
            os = [tokens[j] for j in range(left, right + 1) if j != i]
            for o in os:
                inputs.append(word_to_id[c])
                labels.append(word_to_id[o])

    inputs = np.array(inputs)
    labels = np.array(labels)
    return inputs, labels

In [ ]:
train_inputs, train_labels = make_data(text.split("\n"), 2)
train_inputs, train_labels

## Modeling

In [ ]:
args.n_vocab = len(word_to_id)
args.d_model = 32
args

In [ ]:
def build_model(args):
    """
    skim gram 학습 모델
    :param args
    """
    tokens = tf.keras.layers.Input(shape=(None,))

    # center word vector
    V = tf.keras.layers.Embedding(args.n_vocab, args.d_model)
    v = V(tokens)
    # 단어 예측 (activation=tf.nn.softmax)
    U = tf.keras.layers.Dense(args.n_vocab, use_bias=False, activation=tf.nn.softmax)
    y_pred = U(v)

    model = tf.keras.Model(inputs=tokens, outputs=y_pred)
    return model, V

In [ ]:
model, V = build_model(args)
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

In [ ]:
model.predict(train_inputs)

## embedding 출력

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def plot_embdeeding(embedding, word_to_id):
    """
    word의 embedding vector를 2차원 공간에서 위치를 표현 함
    :param embedding: tf.keras.layers.Embedding 객체
    :param word_to_id: word_to_id vocab
    """
    # font_name = fm.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    # font_name = 'AppleGothic'
    font_name = 'NanumBarunGothic'
    plt.rc('font', family=font_name)
    plt.rcParams["axes.unicode_minus"] = False # 한글 폰트 사용시 - 깨지는 문제 해결

    # word와 vector 값 추출
    vectors = []
    words = []
    for word, id in word_to_id.items():
        if id < 2: continue  # 0: PAD, 1: UNK
        vectors.append(embedding(id).numpy())
        words.append(word)

    # 2차원 보다 큰 경우 PCA를 이용해 2차원으로 차원 축소
    if 2 < len(vectors[0]):
        vectors = PCA().fit_transform(vectors)[:,:2]

    # 벡터와 단어를 화면에 출력
    for word, vector in zip(words, vectors):
        plt.scatter(vector[0], vector[1])
        plt.annotate(word, xy=(vector[0], vector[1]), xytext=(6, 4), textcoords='offset points', ha='right', va='bottom')

    # 출력
    plt.show()

In [ ]:
V = tf.keras.layers.Embedding(len(word_to_id), 4)

plot_embdeeding(V, word_to_id)

## Train

In [ ]:
model, V = build_model(args)
tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
plot_embdeeding(V, word_to_id)

In [ ]:
epochs = 100
# 20번 반복 진행
for i in range(20):
    # epoch 학습 진행 후 vector를 화면에 출력
    model.fit(train_inputs, train_labels, batch_size=512, epochs=epochs, verbose=0)
    print(f"training >>> {(i+1) * epochs}")
    plot_embdeeding(V, word_to_id)

# 실습
- 아래 데이터를 이용해 Word2Vec을 학습해 보세요.

## Data

In [ ]:
# 학습할 말뭉치
text = """현은 과학을 이해하고 좋아한다
현은 수학을 능통하고 좋아한다
현은 영어를 복잡해서 싫어한다
현은 한문을 어려워서 싫어한다
혜웅은 과학을 이해하고 싫어한다
혜웅은 수학을 능통하고 싫어한다
혜웅은 영어를 복잡해서 좋아한다
혜웅은 한문을 어려워서 좋아한다"""

## vocabulary

## Train 데이터 생성

## Modeling

## embedding 출력

## Train

# Gensim
- https://radimrehurek.com/gensim/index.html

In [ ]:
import gensim
import gensim.downloader as api

## Tutorial

In [ ]:
# 이미 학습된 model download
# wv = api.load('word2vec-google-news-300') # 1.6G
wv = api.load('glove-wiki-gigaword-100') # 128M

In [ ]:
# 파일에서 직접 로드
wv = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/문서/강의계획서/삼성전기/삼성전기.20220228/data/glove-wiki-gigaword-100.gz")

In [ ]:
# vocab 개수 및 최초 20개 출력
print(f"len: {len(wv.vocab)}")
for i, word in enumerate(wv.vocab):
    if i >= 20:
        break
    print(f"{i:2d}: {word}")

In [ ]:
wv.most_similar('obama')

In [ ]:
wv.most_similar('banana')

In [ ]:
wv.most_similar('apple')

In [ ]:
# king - man + woman by
result = wv.most_similar(positive=['woman', 'king'], negative=['man'])
result

In [ ]:
def analogy(p1, n1, p2):
    result = wv.most_similar(positive=[p2, p1], negative=[n1])
    return result

In [ ]:
# japanese - japan + australia
analogy('japanese', 'japan', 'australia')

In [ ]:
# beer - australia + france
analogy('beer', 'australia', 'france')

In [ ]:
# clinton - reagan + obama
analogy('clinton', 'reagan', 'obama')

In [ ]:
# tallest - tall + long
analogy('tallest', 'tall', 'long')

In [ ]:
# fantastic - good + bad
analogy('fantastic', 'good', 'bad')

In [ ]:
wv.doesnt_match("breakfast cereal dinner lunch".split())

In [ ]:
def display_pca_scatterplot(model, words=None, n_sample=100):
    if not words:
        words = np.random.choice(list(model.vocab.keys()), n_sample)
    word_vectors = np.array([model[w] for w in words])

    # 폰트
    font_name = "NanumBarunGothic"

    # plot 크기 및 폰트 설정
    plt.rc('font', family=font_name)
    plt.rcParams["axes.unicode_minus"] = False # 한글 폰트 사용시 - 깨지는 문제 해결

    word_vectors = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(12,12))
    plt.scatter(word_vectors[:,0], word_vectors[:,1], edgecolors='k', c='r')
    for word, (x, y) in zip(words, word_vectors):
        plt.text(x, y, word)
    plt.show()

In [ ]:
display_pca_scatterplot(wv, words=['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
                         'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
                         'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
                         'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
                         'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
                         'homework', 'assignment', 'problem', 'exam', 'test', 'class',
                         'school', 'college', 'university', 'institute'])

In [ ]:
display_pca_scatterplot(wv)